In [82]:
import torch
import os
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer

import nemo.collections.asr as nemo_asr
from pathlib import Path


In [83]:
# Load the model from a local .nemo file

MODEL_ROOT = "/external2/karan_exp/experiments/finetune-multidomain-gpt4/2024-07-16_12-46-41/checkpoints/"
MODEL_ROOT = Path(MODEL_ROOT)
model_path = MODEL_ROOT / "finetune-multidomain-gpt4.nemo"

cfg = nemo_asr.models.ASRModel.restore_from(restore_path=model_path, return_config=True)

# If you need to get the configuration, you can access it from the loaded model

print(cfg)

{'sample_rate': 16000, 'log_prediction': True, 'ctc_reduction': 'mean_batch', 'train_ds': {'manifest_filepath': '/home/ksingla/workspace/PromptingNemo/data/synthetic/text_train_cleaned_manifest_v4_0.json', 'sample_rate': 16000, 'batch_size': 12, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'shuffle_n': 2048, 'is_tarred': False, 'tarred_audio_filepaths': None}, 'validation_ds': {'manifest_filepath': '/home/ksingla/workspace/PromptingNemo/data/synthetic/text_valid_cleaned_manifest_v4.json', 'sample_rate': 16000, 'batch_size': 12, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'is_tarred': False, 'tarred_audio_filepaths': 'na'}, 'test_ds': {'manifest_filepath': '/home/ksingla/workspace/PromptingNemo/data/synthetic/text_valid_cleaned_manifest_v4.json', 'sample_rate': 16000, 'batch_size': 12, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 

In [84]:
# from nemo.core import adapter_mixins

# # Utility method to check and update the model config
# def update_model_config_to_support_adapter(model_cfg):
#     with open_dict(model_cfg):
#         adapter_metadata = adapter_mixins.get_registered_adapter(model_cfg.encoder._target_)
#         if adapter_metadata is not None:
#             model_cfg.encoder._target_ = adapter_metadata.adapter_class_path

#     print("Updated encoder _target_ model :", model_cfg.encoder._target_)
#     return model_cfg

In [85]:
cfg = update_model_config_to_support_adapter(cfg)

Updated encoder _target_ model : nemo.collections.asr.modules.conformer_encoder.ConformerEncoderAdapter


In [86]:
model = nemo_asr.models.ASRModel.restore_from(model_path, override_config_path=cfg)

[NeMo I 2024-07-16 20:39:32 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1206 tokens


[NeMo W 2024-07-16 20:39:32 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/ksingla/workspace/PromptingNemo/data/synthetic/text_train_cleaned_manifest_v4_0.json
    sample_rate: 16000
    batch_size: 12
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2024-07-16 20:39:32 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /home/ksingla/workspace/PromptingNemo/data/synthetic/text_valid_cleaned_manif

[NeMo I 2024-07-16 20:39:32 features:289] PADDING: 0
[NeMo I 2024-07-16 20:39:33 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /external2/karan_exp/experiments/finetune-multidomain-gpt4/2024-07-16_12-46-41/checkpoints/finetune-multidomain-gpt4.nemo.


In [87]:
data_dir = "/home/ksingla/workspace/PromptingNemo/data/synthetic/"
TRAIN_MANIFEST = os.path.join(data_dir, "text_train_cleaned_manifest_v5.json")
TEST_MANIFEST = os.path.join(data_dir, "text_valid_cleaned_manifest_v4.json")
ALL_TAGS = os.path.join(data_dir, "alltags_v5.txt")
taglist = []
with open(ALL_TAGS, 'r') as f:
    for line in f:
        word, tag = line.split()
        taglist.append(tag)

In [ ]:
def train_sentencepiece_tokenizer(manifest_file, tokenizer_folder, special_tokens=None, vocab_size=5000):
    # Step 1: Read the manifest file and extract text data
    def read_manifest(manifest_path):
        with open(manifest_path, 'r') as f:
            lines = f.readlines()
        return [json.loads(line)['text'] for line in lines]
    
    text_data = read_manifest(manifest_file)
    
    # Step 2: Save the extracted text to a temporary file
    temp_text_file = os.path.join(tokenizer_folder, 'text_data.txt')
    with open(temp_text_file, 'w') as f:
        for sentence in text_data:
            f.write(sentence + '\n')
    
    # Step 3: Train the SentencePiece tokenizer with special tokens if provided
    model_prefix = os.path.join(tokenizer_folder, 'tokenizer')
    
    # Prepare special tokens string
    if special_tokens:
        user_defined_symbols = ','.join(special_tokens)
        spm.SentencePieceTrainer.train(
            input=temp_text_file, 
            model_prefix=model_prefix, 
            vocab_size=vocab_size,
            user_defined_symbols=user_defined_symbols
        )
    else:
        spm.SentencePieceTrainer.train(
            input=temp_text_file, 
            model_prefix=model_prefix, 
            vocab_size=vocab_size
        )
    
    # Step 4: Return the paths to the tokenizer model and vocab files
    model_file = f"{model_prefix}.model"
    vocab_file = f"{model_prefix}.vocab"
    
    return model_file, vocab_file

In [ ]:
taglist = open(ALL_TAGS, 'r').read().splitlines()
taglist = [tag.split()[1] for tag in taglist]


In [ ]:
train_sentencepiece_tokenizer(TRAIN_MANIFEST, "/home/ksingla/workspace/PromptingNemo/data/synthetic/tokenizer", special_tokens=taglist, vocab_size=5000)

In [88]:
import subprocess
import os
import sentencepiece as spm
import logging
import sys
import json

def generate_sentencepiece_model_pb2(script_dir, proto_file_path):
    # Construct the command
    command = [
        'protoc',
        f'--python_out={script_dir}',
        proto_file_path
    ]
    
    try:
        # Run the command
        subprocess.run(command, check=True)
        print("Successfully generated sentencepiece_model_pb2.py")
    except subprocess.CalledProcessError as e:
        print(f"Error generating sentencepiece_model_pb2.py: {e}")
        sys.exit(1)

def edit_spt_model(input_file, output_folder, tokens, vocab_file, vocab_txt_file, is_userdefined=False):
    from sentencepiece_model_pb2 import ModelProto  # Ensure this import is after the proto generation
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    output_model_file = os.path.join(output_folder, 'tokenizer.model')
    output_vocab_file = os.path.join(output_folder, 'tokenizer.vocab')
    output_vocab_txt_file = os.path.join(output_folder, 'vocab.txt')

    token_type = 3
    if is_userdefined:
        token_type = 4

    model = ModelProto()
    model.ParseFromString(open(input_file, 'rb').read())

    existing_tokens = {piece.piece for piece in model.pieces}

    new_tokens = []
    for token in tokens:
        if token in existing_tokens:
            logging.warning(f"Special Token '{token}' already exists in the input model, skipping.")
            continue
        piece = model.SentencePiece(piece=token, score=0.0, type=token_type)
        model.pieces.append(piece)
        new_tokens.append(token)

    sp = spm.SentencePieceProcessor()
    try:
        sp.LoadFromSerializedProto(model.SerializeToString())
        for token in new_tokens:
            id = sp.piece_to_id(token)
            logging.info(f"Created token '{token}' at ID {id}")
        logging.info(f"New tokenizer vocab size: {sp.get_piece_size()}")
    except:
        logging.error("Could not appropriately configure new tokenizer. Verify if the special tokens already exist.")
        sys.exit(1)

    with open(output_model_file, 'wb') as outf:
        outf.write(model.SerializeToString())

    logging.info(f"Created new tokenizer at: {output_model_file}")

    # Read the original vocab file and append the new tokens
    with open(vocab_file, 'r') as original_vocab_file:
        original_vocab = original_vocab_file.readlines()

    with open(output_vocab_file, 'w') as updated_vocab_file:
        updated_vocab_file.writelines(original_vocab)
        for token in new_tokens:
            updated_vocab_file.write(f"{token}\n")

    # Update vocab.txt
    with open(vocab_txt_file, 'r') as original_vocab_txt_file:
        original_vocab_txt = original_vocab_txt_file.readlines()

    with open(output_vocab_txt_file, 'w') as updated_vocab_txt_file:
        updated_vocab_txt_file.writelines(original_vocab_txt)
        for token in new_tokens:
            updated_vocab_txt_file.write(f"{token}\n")

    logging.info(f"Updated vocab files: {output_vocab_file}, {output_vocab_txt_file}")

def update_model_config(model, new_model_path):
    model['cfg']['tokenizer']['model_path'] = new_model_path
    logging.info(f"Updated model configuration with new tokenizer model path: {new_model_path}")


# Define input and output paths
input_folder = MODEL_ROOT / "tokenizer"
output_folder = MODEL_ROOT / "new_tokenizer"


#input_folder = '/external/ksingla/models/nemo/stt_en_conformer_ctc_small/tokenizer'
#output_folder = '/external/ksingla/models/nemo/stt_en_conformer_ctc_small/new_tokenizer'
#proto_dir = '/path/to/save/proto'  # Define the actual path where the proto file should be saved
#proto_file = '/path/to/sentencepiece_model.proto'  # Define the actual path to the sentencepiece_model.proto file

input_file = input_folder / 'tokenizer.model'
vocab_file = input_folder / 'tokenizer.vocab'
vocab_txt_file = input_folder / 'vocab.txt'

# input_file = os.path.join(input_folder, 'tokenizer.model')
# vocab_file = os.path.join(input_folder, 'tokenizer.vocab')
# vocab_txt_file = os.path.join(input_folder, 'vocab.txt')

# Include all single-digit integers in the tokens list
punctuations = ['.', ',', '?', '!', ';', ':', '-', '(', ')', '[', ']', '{', '}', '<', '>', '/', '\\', '|', '@', '#', '$', '%', '^', '&', '*', '+', '=', '~', '`', '_', '"', "'"]
tokens = taglist + [str(i) for i in range(10)] + punctuations
is_userdefined = True

# Step 1: Generate the sentencepiece_model_pb2.py file
#generate_sentencepiece_model_pb2(proto_dir, proto_file)

# Step 2: Edit the SentencePiece model
edit_spt_model(input_file, output_folder, tokens, vocab_file, vocab_txt_file, is_userdefined)

# Step 3: Load the model configuration and update it
# model_config_file = '/path/to/model/config.json'  # Define the actual path to the model config file
# with open(model_config_file, 'r') as f:
#     model = json.load(f)

# new_model_path = os.path.join(output_folder, 'tokenizer.model')
# update_model_config(model, new_model_path)

# # Save the updated model configuration
# with open(model_config_file, 'w') as f:
#     json.dump(model, f, indent=4)

#logging.info(f"Updated model configuration saved to: {model_config_file}")


Special Token 'T0' already exists in the input model, skipping.
Special Token 'T1' already exists in the input model, skipping.
Special Token 'T2' already exists in the input model, skipping.
Special Token 'T3' already exists in the input model, skipping.
Special Token 'T4' already exists in the input model, skipping.
Special Token 'T5' already exists in the input model, skipping.
Special Token 'T6' already exists in the input model, skipping.
Special Token 'T7' already exists in the input model, skipping.
Special Token 'T8' already exists in the input model, skipping.
Special Token 'T9' already exists in the input model, skipping.
Special Token 'T10' already exists in the input model, skipping.
Special Token 'T11' already exists in the input model, skipping.
Special Token 'T12' already exists in the input model, skipping.
Special Token 'T13' already exists in the input model, skipping.
Special Token 'T14' already exists in the input model, skipping.
Special Token 'T15' already exists 

Special Token 'T283' already exists in the input model, skipping.
Special Token 'T284' already exists in the input model, skipping.
Special Token 'T285' already exists in the input model, skipping.
Special Token 'T286' already exists in the input model, skipping.
Special Token 'T287' already exists in the input model, skipping.
Special Token 'T288' already exists in the input model, skipping.
Special Token 'T289' already exists in the input model, skipping.
Special Token 'T290' already exists in the input model, skipping.
Special Token 'T291' already exists in the input model, skipping.
Special Token 'T292' already exists in the input model, skipping.
Special Token 'T293' already exists in the input model, skipping.
Special Token 'T294' already exists in the input model, skipping.
Special Token 'T295' already exists in the input model, skipping.
Special Token 'T296' already exists in the input model, skipping.
Special Token 'T297' already exists in the input model, skipping.
Special To

In [89]:
model.change_vocabulary(output_folder, "bpe")

[NeMo W 2024-07-16 20:39:34 modelPT:258] You tried to register an artifact under config key=tokenizer.model_path but an artifact for it has already been registered.
[NeMo W 2024-07-16 20:39:34 modelPT:258] You tried to register an artifact under config key=tokenizer.vocab_path but an artifact for it has already been registered.
[NeMo W 2024-07-16 20:39:34 modelPT:258] You tried to register an artifact under config key=tokenizer.spe_tokenizer_vocab but an artifact for it has already been registered.


[NeMo I 2024-07-16 20:39:34 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1271 tokens
[NeMo I 2024-07-16 20:39:34 ctc_bpe_models:248] 
    Replacing old number of classes (1206) with new number of classes - 1271
[NeMo I 2024-07-16 20:39:35 ctc_bpe_models:290] Changed tokenizer to ['<unk>', 's', '▁', 'e', 't', 'u', 'd', 'a', 'o', 'n', 'i', '▁the', '▁a', 'm', 'y', 'l', 'h', 'p', 're', '▁s', 'g', 'r', '▁to', '▁i', 'ing', '▁and', 'f', '▁p', 'an', 'c', 'w', 'er', 'ed', '▁of', '▁in', 'k', "'", '▁w', 'ar', 'or', '▁f', 'b', '▁b', 'en', '▁you', 'al', 'le', 'in', 'll', '▁that', '▁he', 'ro', '▁t', 'es', '▁it', '▁be', 've', 'v', 'ly', '▁c', 'th', '▁o', 'ent', 'ch', 'ur', '▁we', '▁re', '▁n', 'it', '▁so', '▁co', '▁g', '▁on', '▁for', 'on', 'ce', 'ri', '▁do', '▁is', '▁ha', '▁ma', 'ver', 'li', 'ra', '▁was', 'ic', 'la', '▁e', 'se', 'ter', 'ct', 'ion', '▁ca', '▁st', '▁me', 'ir', '▁mo', '▁with', '▁but', '▁have', '▁go', '▁de', '▁ho', '▁di', '▁not', '▁know', '▁lo', '▁this', 'ation', 'ther', 

In [90]:
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
max_steps = 600000

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=50, check_val_every_n_epoch=1, accumulate_grad_batches=8)

model.set_trainer(trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [91]:
# utility method
import json
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest

In [92]:
with open_dict(model.cfg):
  # Train Dataloader
  model.cfg.train_ds.manifest_filepath = TRAIN_MANIFEST
  model.cfg.train_ds.batch_size = 12
  model.cfg.train_ds.is_tarred = False
  model.cfg.train_ds.tarred_audio_filepaths = None
  model.cfg.train_ds.num_workers = 8  # Adding num_workers for training dataloader

  model.cfg.validation_ds.manifest_filepath = TEST_MANIFEST
  model.cfg.validation_ds.batch_size = 12
  model.cfg.validation_ds.num_workers = 8  # Adding num_workers for training dataloader

model.setup_training_data(model.cfg.train_ds)
model.setup_multiple_validation_data(model.cfg.validation_ds)
model.setup_multiple_test_data(model.cfg.validation_ds)

[NeMo I 2024-07-16 20:39:36 collections:196] Dataset loaded with 24542 files totalling 30.86 hours
[NeMo I 2024-07-16 20:39:36 collections:197] 2 files were filtered totalling 0.01 hours
[NeMo I 2024-07-16 20:39:36 collections:196] Dataset loaded with 2188 files totalling 2.47 hours
[NeMo I 2024-07-16 20:39:36 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-16 20:39:36 collections:196] Dataset loaded with 2188 files totalling 2.47 hours
[NeMo I 2024-07-16 20:39:36 collections:197] 0 files were filtered totalling 0.00 hours


In [93]:
with open_dict(model.cfg):
  # Spec Augment
  model.cfg.spec_augment.freq_masks = model.cfg.spec_augment.freq_masks  # Can be changed
  model.cfg.spec_augment.freq_width = model.cfg.spec_augment.freq_width  # Can be changed
  model.cfg.spec_augment.time_masks = model.cfg.spec_augment.time_masks  # Can be changed
  model.cfg.spec_augment.time_width = model.cfg.spec_augment.time_width  # Can be changed

model.spec_augmentation = model.from_config_dict(model.cfg.spec_augment)

In [94]:
if 'optim' in model.cfg:
  print(OmegaConf.to_yaml(model.cfg.optim))

name: adamw
lr: 0.1
betas:
- 0.9
- 0.98
weight_decay: 0.0001
sched:
  name: NoamAnnealing
  d_model: 512
  warmup_steps: 1000
  warmup_ratio: null
  min_lr: 1.0e-06



In [95]:
with open_dict(model.cfg):
  model.cfg.optim.lr = 0.1
  model.cfg.optim.weight_decay = 0.0001
  model.cfg.optim.sched.warmup_steps = 100

model.setup_optimization(model.cfg.optim);

[NeMo I 2024-07-16 20:39:36 modelPT:723] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0001
    )
[NeMo I 2024-07-16 20:39:36 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x7c50113cb640>" 
    will be used during training (effective maximum steps = 600000) - 
    Parameters : 
    (d_model: 512
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 600000
    )


In [96]:
# if hasattr(model, 'adapter_module_names'):
#   print(model.adapter_module_names)

In [97]:
# for module in model.children():
#   if hasattr(module, 'get_accepted_adapter_types'):
#     types = module.get_accepted_adapter_types()
#     print("Module : ", module.__class__.__name__)

#     for tp in types:
#       print(tp)
#     print()

In [98]:
# from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig

In [99]:

# adapter_name = "AN4" #@param {type:"string"}
# adapter_dim = 32 #@param {type:"integer"}
# adapter_activation = "swish" #@param {type:"string"}
# adapter_norm_position = "pre" #@param ["pre", "post"]

In [100]:
# adapter_cfg = LinearAdapterConfig(
#     in_features=model.cfg.encoder.d_model,  # conformer specific model dim. Every layer emits this dim at its output.
#     dim=adapter_dim,  # the bottleneck dimension of the adapter
#     activation=adapter_activation,  # activation used in bottleneck block
#     norm_position=adapter_norm_position,  # whether to use LayerNorm at the beginning or the end of the adapter
# )
# print(adapter_cfg)

In [101]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 121 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 652 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
488.351   Total estimated model params size (MB)

In [102]:
# model.add_adapter(name=adapter_name, cfg=adapter_cfg)

In [103]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 121 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 652 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
488.351   Total estimated model params size (MB)

In [104]:
# model.set_enabled_adapters(enabled=False)  # disable all adapters
# model.set_enabled_adapters(name=adapter_name, enabled=True)  # enable only the current adapter we want to train

In [105]:
# model.freeze()
# model.unfreeze_enabled_adapters()
# #model.unfreeze()
# model.decoder.unfreeze()

In [106]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 121 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 652 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
488.351   Total estimated model params size (MB)

In [107]:
# Prepare NeMo's Experiment manager to handle checkpoint saving and logging for us
from nemo.utils import exp_manager

# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'/external2/karan_exp/experiments/',
    name=f"finetune-multidomain-gpt-v5",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2024-07-16 20:39:38 exp_manager:396] Experiments will be logged at /external2/karan_exp/experiments/finetune-multidomain-gpt-v5/2024-07-16_20-39-38
[NeMo I 2024-07-16 20:39:38 exp_manager:842] TensorboardLogger has been set up


[NeMo W 2024-07-16 20:39:38 exp_manager:952] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 600000. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


In [108]:
model.cfg['tokenizer']



{'dir': PosixPath('/external2/karan_exp/experiments/finetune-multidomain-gpt4/2024-07-16_12-46-41/checkpoints/new_tokenizer'), 'type': 'bpe', 'model_path': '/external2/karan_exp/experiments/finetune-multidomain-gpt4/2024-07-16_12-46-41/checkpoints/new_tokenizer/tokenizer.model', 'vocab_path': '/external2/karan_exp/experiments/finetune-multidomain-gpt4/2024-07-16_12-46-41/checkpoints/new_tokenizer/vocab.txt', 'spe_tokenizer_vocab': '/external2/karan_exp/experiments/finetune-multidomain-gpt4/2024-07-16_12-46-41/checkpoints/new_tokenizer/tokenizer.vocab'}

In [109]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 121 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 652 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
488.351   Total estimated model params size (MB)

In [110]:
# Finally, train the adapters
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-07-16 20:39:38 modelPT:723] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0001
    )
[NeMo I 2024-07-16 20:39:38 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x7c50110b8a90>" 
    will be used during training (effective maximum steps = 600000) - 
    Parameters : 
    (d_model: 512
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 600000
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 121 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 652 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
488.351   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo I 2024-07-16 20:39:40 wer:318] 
    
[NeMo I 2024-07-16 20:39:40 wer:319] reference:T3-T826-T248 T1_T248_T668 i T0 T1_T201 am experiencing T0 T1_T667 shortness of breath T0 T1_T54 lately T0 T2_T667_T47
[NeMo I 2024-07-16 20:39:40 wer:320] predicted:T1001T1039icT458T955T663T975T235T305T248T305zT180T563T756T1177T305T449T775 likeT778T396T463T305T903T840 notT162T305zT396T149T518T207T661T351T398T223T1001T203T710T407T203 notT305T731T858T264T983T90T407T235T855T8T991T913T666T396
[NeMo I 2024-07-16 20:39:40 wer:318] 
    
[NeMo I 2024-07-16 20:39:40 wer:319] reference:T3-T826-T248 T1_T248_T668 i T0 T1_T201 underwent T0 T1_T225 a knee replacement surgery T0 T1_T623 six months ago T0 T2_T225_T47
[NeMo I 2024-07-16 20:39:40 wer:320] predicted:T1001 uhT1073T169T663thT235T479T248T220T442T592T672T840 notT285T991 ⁇ T631T181T800T285T135in gT840T80T305T849T413T1279T183T181T183T220T413T1172T648T1172T1341T672T967T396T1228T903T800T384T817T384T410T672T11T492T840T357T305T444T134T90T490T607T355T891T437T

...

[NeMo I 2024-07-16 23:42:37 nemo_model_checkpoint:177] New best .nemo model saved to: /external2/karan_exp/experiments/finetune-multidomain-gpt-v5/2024-07-16_20-39-38/checkpoints/finetune-multidomain-gpt-v5.nemo
[NeMo I 2024-07-16 23:42:58 wer:318] 
    
[NeMo I 2024-07-16 23:42:58 wer:319] reference:T3-T33-T1023-T511 T1_T201 pause T0 my T1_T682_T308 current strength training session end. T2_T150_T345
[NeMo I 2024-07-16 23:42:58 wer:320] predicted:T3-T33-T1023-T511 T1_T201 pause T0 my T1_T345_T485 currentT308 strength trainingT0 session end. T2_T150_T345
[NeMo I 2024-07-16 23:43:17 wer:318] 
    
[NeMo I 2024-07-16 23:43:17 wer:319] reference:T3-T33-T1023-T511 T1_T201 record T0 my T1_T282_T318 weight T0 every T1_T297 morning end. T2_T168_T282
[NeMo I 2024-07-16 23:43:17 wer:320] predicted:T3-T33-T1023-T511 T1_T201 record T0 my T1_T282_T343 weight T0 every T1_T297 morning end. T2_T168_T282
[NeMo I 2024-07-16 23:43:37 wer:318] 
    
[NeMo I 2024-07-16 23:43:38 wer:319] reference:T3-T26-T

Validation: 0it [00:00, ?it/s]

[NeMo I 2024-07-16 23:56:10 wer:318] 
    
[NeMo I 2024-07-16 23:56:10 wer:319] reference:T3-T826-T248 T1_T248_T668 i T0 T1_T201 am experiencing T0 T1_T667 shortness of breath T0 T1_T54 lately T0 T2_T667_T47
[NeMo I 2024-07-16 23:56:10 wer:320] predicted:T3-T826-T248 T1_T248_T668 i T0 T1_T201 a experiencing T0 T1_T667 shortness of breath T0 T1_T297 lately T0 T2_T667_T50
[NeMo I 2024-07-16 23:56:10 wer:318] 
    
[NeMo I 2024-07-16 23:56:10 wer:319] reference:T3-T826-T248 T1_T248_T668 i T0 T1_T201 underwent T0 T1_T225 a knee replacement surgery T0 T1_T623 six months ago T0 T2_T225_T47
[NeMo I 2024-07-16 23:56:10 wer:320] predicted:T3-T826-T248 T1_T248_T668 i T0 T1_T201 underwent T0 T1_T225 a ne replacement surgery T0 T1_T623 six months ago T0 T2_T225_T50
[NeMo I 2024-07-16 23:56:10 wer:318] 
    
[NeMo I 2024-07-16 23:56:10 wer:319] reference:T3-T826-T248 T1_T248_T668 i T0 T1_T201 have been advised to T0 T1_T201 monitor T0 my T1_T667 weight T0 T1_T297 weekly T0 T2_T208_T643
[NeMo I 2024

Epoch 13, global step 3584: 'val_wer' reached 0.22316 (best 0.22316), saving model to '/external2/karan_exp/experiments/finetune-multidomain-gpt-v5/2024-07-16_20-39-38/checkpoints/finetune-multidomain-gpt-v5--val_wer=0.2232-epoch=13.ckpt' as top 3


[NeMo I 2024-07-16 23:56:42 nemo_model_checkpoint:177] New best .nemo model saved to: /external2/karan_exp/experiments/finetune-multidomain-gpt-v5/2024-07-16_20-39-38/checkpoints/finetune-multidomain-gpt-v5.nemo
[NeMo I 2024-07-16 23:57:04 wer:318] 
    
[NeMo I 2024-07-16 23:57:04 wer:319] reference:T3-T461-T511 T1_T201 remove T0 T1_T495 one T0 T1_T421 gallon of milk T0 from my T1_T660_T308 shopping list T0 T2_T301_T519_T660
[NeMo I 2024-07-16 23:57:04 wer:320] predicted:T3-T461-T511 T1_T201 remove T0 T1_T4951 T0 T1_T421 gallon ofT421 milk T0 from my T1_T660_T308 shopping list T0 T2_T301_T519_T660
[NeMo I 2024-07-16 23:57:23 wer:318] 
    
[NeMo I 2024-07-16 23:57:23 wer:319] reference:T3-T887-T162 T1_T536_T472 modric T0 pulls the strings in midfield, dictating the tempo of the game with his precise T1_T201 passing T0 and vision. T2_T894_T308
[NeMo I 2024-07-16 23:57:23 wer:320] predicted:T3-T887-T162 T1_T536_T472  momkp pols the strings in theeieldd, deating the  tempou of the game w